In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Thu Feb 16 10:44:47 2017

@author: whikwon
"""

import pandas as pd
import os

#폴더 지정
os.chdir('//Client/D$/#.Secure Work Folder/2.투입실적/17년 투입 실적/연태')

#불량명 번역
defect_name = {'圆形':'원형','未':'미','贴附':'부착','凸起':'돌기','压痕':'눌림','分层':'층간박리','程度':'정도',
               '不定型压痕':'부정형눌림','POL缺失':'Pol파손','压印':'눌림','异物':'이물','缺胶':'점착제빠짐','撞角':'꺾임',
               '划伤':'정도','放反':'뒤집힘','程度不良':'정도','割伤':'S/C','延伸压痕':'MD눌림','气泡':'기포','折伤':'꺾임',
               'PNL':'패널','破损':'깨짐','红线':'홍선','唇型斑纹':'입술얼룩','唇斑':'입술얼룩','粘着剂':'점착제빠짐',
               '紫斑':'보라색얼룩','粘胶剂':'점착제빠짐','白线':'백선','脏污':'Dirty','斑纹':'얼룩','不良':'불량','延伸':'연신'}
               
check_result = {'圆形':'원형','未':'미','贴附':'부착','凸起':'돌기','压痕':'눌림','分层':'층간박리','程度':'정도',
                '不定型压痕':'부정형눌림','POL缺失':'Pol파손','压印':'눌림','异物':'이물','缺胶':'점착제빠짐','撞角':'꺾임',
                '划伤':'정도','放反':'뒤집힘','程度不良':'정도','割伤':'S/C','延伸压痕':'MD눌림','气泡':'기포','折伤':'꺾임',
                'PNL':'패널','破损':'깨짐','红线':'홍선','唇型斑纹':'입술얼룩','唇斑':'입술얼룩','粘着剂':'점착제빠짐',
                '紫斑':'보라색얼룩','粘胶剂':'점착제빠짐','白线':'백선','脏污':'Dirty','斑纹':'얼룩','延伸':'연신',
                '不良':'불량','资材':'자재','无':'무','关':'관','保护膜':'보호필름','有':'유','现象':'현상','同一':'동일','角':'모서리','部':'부',
                '去':'제거','后':'후','造成':'유발','好':'잘','胶':'점착제','撞伤':'부딪힌','痕迹':'흔적','线状':'선형','单品':'단품',
                '确认':'확인','末端':'단말','强制':'강제','执行':'실행','表面':'표면','撞击':'부딪힌','设备':'설비','修理品':'수리품',
                '卡的':'부딪힘','在产线':'라인내','线头':'실오라기','起始':'시작','位置':'위치','以内':'이내','扒过':'박리','撞':'데미지'}

In [ ]:
#데이터 불러오기
defect_data = pd.read_excel('//Client/D$/#.Secure Work Folder/2.투입실적/17년 투입 실적/연태/2017-04-24 Daily Report -.xlsx', sheetname = '复判不良', parse_cols = ('C:AK'))
input_data = pd.read_excel('//Client/D$/#.Secure Work Folder/2.투입실적/17년 투입 실적/연태/2017-04-24 Daily Report -.xlsx', sheetname = '投入数量', parse_cols = ('B:L'))

In [ ]:
#defect_data 수정
defect_data.columns = ['투입일','투입시간','Site','Model','상/하','Part-no','불량명','불량수량','불량Lot','LGC 미귀책 사유','확인자(CS)','장소','원형눌림','PIT','MD눌림','부정형눌림','보라색얼룩',
                '눌림','돌기','적색돌기','기포','미부착','정도','패널 깨짐','꺾음','S/C','얼룩','입술얼룩','APF층간박리','HC층간박리','꺾임','점착제빠짐','이물','홍선','기타']                

In [ ]:
#투입 일자 
defect_data['투입일'] = defect_data['투입일'].apply(lambda x: pd.Timestamp('1899-12-30') + pd.Timedelta(days = x))
defect_data.index = defect_data['투입일']
defect_data['Model'] = defect_data['Model'].apply(lambda x: x.upper())
defect_data['투입월'] = defect_data['투입일'].apply(lambda x: x.month)
defect_data['투입일자'] = defect_data['투입일'].apply(lambda x: x.day)

In [ ]:
#번역 진행 
defect_data['불량명'] = defect_data['불량명'].replace(defect_name, regex = True)
defect_data['LGC 미귀책 사유'] = defect_data['LGC 미귀책 사유'].replace(check_result, regex = True)
defect_data['LGC 미귀책 사유'].fillna('-', inplace = True)
defect_data['불량Lot'] = defect_data['불량Lot'].replace({'不承认':'해당없음'}, regex = True)
defect_data['불량Lot'].fillna('미확인')
defect_data['상/하'] = defect_data['상/하'].replace({'上':'상','下':'하'}, regex = True)

In [ ]:
#Input_data 수정
input_data.dropna(how = 'all', inplace = True)
input_data.columns = ['투입일','투입시간','투입라인','Site','Part-no','Model','제품명','Lot no.','?','투입수량','비고']
input_data.dropna(subset = ['Model'], inplace = True)
input_data['투입일'] = input_data['투입일'].apply(lambda x: pd.Timestamp('1899-12-30') + pd.Timedelta(days = x))
input_data['Model'] = input_data['Model'].apply(lambda x: x.upper())
input_data['투입월'] = input_data['투입일'].apply(lambda x: x.month)
input_data['투입일자'] = input_data['투입일'].apply(lambda x: x.day)
input_data.index = input_data['투입일']

In [ ]:
#데이터 합치기
defect_summary = defect_data[['투입월','투입일자','Model','Part-no','상/하','불량명','불량수량','LGC 미귀책 사유','불량Lot']]
input_summary = input_data.groupby(['Part-no','투입월'])[['투입수량']].sum().reset_index()
data = pd.merge(defect_summary, input_summary, how = 'left', on = ['Part-no','투입월'])

In [ ]:
#오늘 발생 불량
today_data = data[(data['투입일자'] == 20)&(data['투입월'] == 2)].groupby(['Model','Part-no','상/하','투입월','투입수량','투입일자','불량명','LGC 미귀책 사유'])[['불량수량']].sum()

In [ ]:
#Data 저장하기
writer = pd.ExcelWriter('YT Defect 정리.xlsx')
today_data.to_excel(writer, 'Sheet1')
data.to_excel(writer, 'Sheet2', index = False)
writer.save()